In [1]:
import pyforest
from rdkit import Chem
from rdkit.Chem import AllChem
from imblearn.over_sampling import SMOTEN, ADASYN, SMOTE 

In [2]:
data_path = '../datasets/original_datasets/'
test = pd.read_csv(data_path+'NEK2_1_uM_min_50_pct_binding.csv')
test

<IPython.core.display.Javascript object>

,target_name,compound_id,base_rdkit_smiles,pct_binding,active
0,NEK2,kdb_10,CCS(=O)(=O)N1CC(CC#N)(n2cc(-c3ncnc4[nH]ccc34)c...,3.225806,0
1,NEK2,kdb_100,C[C@@H](Oc1cc(C(=O)Nc2ccc(C(=O)N3CCN(C)CC3)cc2...,3.225806,0
2,NEK2,kdb_1003,CC1(O)CC(c2nc(-c3ccc4ccc(-c5ccccc5)nc4c3)c3c(N...,5.000000,0
3,NEK2,kdb_101,COCC(=O)NC/C=C/c1ccc2ncnc(Nc3ccc(Oc4ccc(C)nc4)...,6.158358,0
4,NEK2,kdb_102,COC(=O)c1ccc2c(c1)NC(=O)/C2=C(\Nc1ccc(N(C)C(=O...,6.158358,0
...,...,...,...,...,...
1403,NEK2,kdb_976,Oc1c2ccccc2c2c3c(cccc13)N=N2,9.090909,0
1404,NEK2,kdb_98,Cc1nc(Nc2n[nH]c3c2CN(C(=O)N[C@H](CN(C)C)c2cccc...,3.225806,0
1405,NEK2,kdb_980,Cc1cnc(-c2cnc(NCCNc3ccc(C#N)cn3)nc2-c2ccc(Cl)c...,85.000000,1
1406,NEK2,kdb_986,Nc1n[nH]c2nnc(-c3ccccc3)c(-c3ccccc3)c12,0.000000,0


In [ ]:
def check_labels(data_path, filename): 
  
    df = pd.read_csv(data_path+filename)

    pct_col = [col for col in df.columns if col.startswith('pct_')] 
    len(pct_col) == 1 
    pct_col = pct_col[0]
    invalid_labels = df[df['active']!= (df[pct_col]>=50).astype(int)]
    if invalid_labels.empty: 
        # print(f'correct label assignment based on {pct_col} column')
        pass
    else: 
        print(f'Error on label assignement.')
        print(invalid_labels)
    # check for duplicates here 
    duplicates = df[df.duplicated()]
    print(f'{df.shape}')
    #duplicates = df[df.duplicated(subset=['base_rdkit_smiles'])]
   
    if not duplicates.empty:
        print('Duplicate rows found:')
        print(duplicates)
        df_cleaned = df.drop_duplicates()
        print(f'df size original: {df.shape}')
        print(f'remove duplicates. {df_cleaned.shape}')
check_labels(data_path,'NEK2_1_uM_min_50_pct_inhibition.csv')

In [ ]:
#create a dataset with folds

# must specify filename because the 50pct binding and inhibition files differ
# to-do: discuss how to do this --> this will create a file that is just this but with column specifying split
#       do we want to already integrate a split_uuid like AMPL? 
#       additionally, we still have to use the splits to create the "scaled df" 
def remove_duplicates(datapath, filename):
    df = pd.read_csv(f'{datapath}{filename}')
    duplicates = df.duplicated(subset=['base_rdkit_smiles'])
    if duplicates.any():
        print('Duplicate rows found:')
        print(df[duplicates])
        df_cleaned = df.drop_duplicates(subset=['base_rdkit_smiles'])
        print(f'df size original: {df.shape}')
        print(f'Removed duplicates. New df size: {df_cleaned.shape}')
    else: 
        df_cleaned = df
    return df_cleaned
    
def create_folds(datapath, filename, k_splits): 
    """filename: NEK#_1_uM_min_50_pct_(binding or inhibition).csv"""
    df = remove_duplicates(datapath, filename)
    # is this extra/too much and should we just assume what labels coincide with majority and minority? 
    # determine majority/minority class 
    #       AMPL takes in 'response column' 
    class_labels = df['active'].value_counts() 
    print(class_labels)
    if len(class_labels)>1: 
        majority_class_label =class_labels.idxmax() 
        majority_num = class_labels.max() 
        minority_class_label = class_labels.idxmin()
        minority_count = class_labels.min()
    df_majority = df[df['active']==majority_class_label]
    df_minority=df[df['active']==minority_class_label]
    # copy to avoid warnings 
    df_majority = df_majority.copy()
    df_minority = df_minority.copy()
    kf = KFold(k_splits,shuffle=True, random_state=42)
    # majority 
    for i, (_, v_ind) in enumerate(kf.split(df_majority)):
        df_majority.loc[df_majority.index[v_ind], 'fold'] = f"fold{i+1}"
    # minority 
    for i, (_, v_ind) in enumerate(kf.split(df_minority)):
        df_minority.loc[df_minority.index[v_ind], 'fold'] = f"fold{i+1}"
    all_fold_df = pd.concat([df_majority,df_minority])
    
    print(all_fold_df.shape)
    print(all_fold_df.active.value_counts())
    ## actually, this might be perfect jsut to save as "split df" because then 
    # in the next step of creating a dataset, you can choose what fold you want to be the test set 
    return all_fold_df

In [ ]:
def train_and__(data_path, fold_df, test_fold):
   """function to label 'train' or 'test' in the 'subset' column
   to be used to create train/test OR train/val
   fold_df: dataframe with column 'fold'
   test_fold (str): fold to make the test set (the remaining folds will be train)
   """
   df = pd.read_csv(data_path+fold_df) 
   df['subset'] = df['fold'].apply(lambda x: 'test' if x == test_fold else 'train')

   return df[['NEK','compound_id', 'active', 'base_rdkit_smiles', 'subset']]

In [ ]:
def get_bind_inhib(df_name):
    match = re.search(r'_([^_]+)\.csv$',df_name)
    return match.group(1) if match else None
    # returns 'binding' or 'inhibition' 
og_dfs = [file for file in os.listdir(data_path) if file.startswith('NEK')]

for df in og_dfs: 
    folded_df = create_folds(data_path, df,5)
    bind_inhib = get_bind_inhib(df)
    nek_name = f'NEK{df[3]}_{bind_inhib}'
    print(nek_name)
    folded_df['NEK']=nek_name 
    folded_df.to_csv(f'../datasets/initial_fold/{nek_name}_initial_fold.csv',index=False)

In [ ]:
neks = ['NEK2_binding', 'NEK2_inhibition', 'NEK3_binding', 'NEK5_binding','NEK9_binding','NEK9_inhibition']

for nek in neks:
    fold_df_name = nek+"_initial_fold.csv"
    print(fold_df_name)
    labeled_df = train_and__("/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/initial_fold/" , fold_df_name, "fold1")
    labeled_df.to_csv(f'/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/{nek}_80_20_df.csv',index=False)

In [ ]:
#featurize 
def featurize(data_path, filename, feat_type, moe_path=None, moe_file=None, mfp_radius=2, nBits=2048): 
    if (feat_type == 'MOE') and (moe_path is not None) and (moe_file is not None): 
        df = create_moe(data_path, filename, moe_path, moe_file)
    elif feat_type == 'MFP': 
        df = create_mfp(data_path, filename, mfp_radius, nBits)
    
    return df


In [ ]:
def create_moe(data_path, filename, moe_path, moe_file):
    """(intended use for already existing dataset)
    This function will use an existing dataframe with smiles column to
    get the features from an existing file (moe_path+moe_file) with the MOE features generated"""
    
    drop_cols = ['active', 'compound_id']
    df = remove_duplicates(data_path, filename)
    df=df.drop(columns=drop_cols)
    

    moe_df=remove_duplicates(moe_path,moe_file)
    

    final_df=moe_df.merge(df, how='outer', on=['base_rdkit_smiles'], suffixes=('_moe_desc', '_og'))
    id_cols = ['NEK', 'compound_id','base_rdkit_smiles','subset', 'active']
    feat_cols = set(list(final_df.columns))-set(id_cols)
    
    final_order_cols = list(id_cols)+list(feat_cols)


    return final_df[final_order_cols]
    # in production code , check for smiles col 

test_df = featurize('/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/','NEK2_inhibition_80_20_df.csv', 'MOE','/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/original_datasets/scaled_descriptors/' , 'NEK2_1_uM_min_50_pct_inhibition_with_moe_descriptors.csv')

In [ ]:
def smiles_to_fps(smiles_list, radius=2, nBits=2048):
    fps = []
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
            arr = np.zeros((1,), dtype=np.int8)
            Chem.DataStructs.ConvertToNumpyArray(fp, arr)
            fps.append(arr)
    return np.array(fps)


def create_mfp(file_path, filename, mfp_radius, nBits):
    df = pd.read_csv(file_path+filename)

    id_data = df[['compound_id','base_rdkit_smiles', 'NEK','subset','active']]
    smiles = df['base_rdkit_smiles']

    mfp_feats = smiles_to_fps(smiles,mfp_radius,nBits)
    mfp_df = pd.DataFrame(mfp_feats)

    id_cols = ['NEK', 'compound_id','base_rdkit_smiles','subset', 'active']
    feat_cols = set(list(mfp_df.columns))-set(id_cols)
    final_order_cols = list(id_cols)+list(feat_cols)
    final_df = pd.concat([mfp_df, df],axis=1)
    final_df = final_df[final_order_cols]
    #display(final_df)

    
    # testX_mfp_df['active'] = test_y.reset_index(drop=True)
    # final_df = pd.concat([trainX_mfp_df,testX_mfp_df], ignore_index=True)
    return final_df
        

test_df = featurize('/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/','NEK2_inhibition_80_20_df.csv', 'MFP',2,2048 )

In [ ]:
og_df_path = '/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/' 
scaled_desc_path = '/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/original_datasets/scaled_descriptors/'
featurized_filepath = '/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/' 
for nek in ['2','3','5','9']: 
    if nek in ['2','9']:
        bind_inhib = ['binding','inhibition']
    else: 
        bind_inhib = ['binding']
    for bi in bind_inhib: 

        for feat in ['MOE','MFP']: 
            print(f'NEK {nek} {bi} {feat}')
            og_df_name = f'NEK{nek}_{bi}_80_20_df.csv'
            if feat == 'MOE': 
                desc_filename =f'NEK{nek}_1_uM_min_50_pct_{bi}_with_moe_descriptors.csv'
                final_moe_df = featurize(og_df_path,og_df_name,feat,scaled_desc_path,desc_filename)
                final_moe_df.to_csv(f'{featurized_filepath}NEK{nek}_{bi}_{feat}_none_unscaled.csv',index=False)
                
            elif feat=='MFP': 
                final_mfp_df = featurize(og_df_path,og_df_name,feat,2,2048)
                final_mfp_df.to_csv(f'{featurized_filepath}NEK{nek}_{bi}_{feat}_none_unscaled.csv',index=False)



In [ ]:
#creating scalers for eack Nek MOE feats
id_cols = ['NEK', 'compound_id','base_rdkit_smiles','subset', 'active']

featurized_filepath = '/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/' 
scaler_dir = '/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/scalers/'
for nek in neks:  
   for feat in ['MOE']: 
      
        df = pd.read_csv(f'{featurized_filepath}{nek}_{feat}_none_unscaled.csv')
        feat_cols = set(list(df.columns))-set(id_cols)
        train_x = df[df["subset"] == "train"]
        train_id = train_x[id_cols]
        train_x.drop(columns = id_cols, inplace=True)
        test_x = df[df["subset"] == "test"]
        test_id = test_x[id_cols]
        test_x.drop(columns = id_cols, inplace=True)
        
        scaling = StandardScaler()
        scaling.fit(train_x)
        #save to pickle
        with open(f'{scaler_dir}_{nek}_{feat}_scaler.pkl', 'wb') as f: 
            pickle.dump(scaling, f) 
        train_scaled = scaling.transform(train_x)
        test_scaled = scaling.transform(test_x)
        trainX_final = pd.DataFrame(train_scaled, columns=list(feat_cols))
        testX_final = pd.DataFrame(test_scaled, columns=list(feat_cols))
        scaled_data = pd.concat([trainX_final, testX_final])
        id_data = pd.concat([train_id,test_id])
        no_samp_data_scaled = pd.concat([id_data.reset_index(drop=True), scaled_data.reset_index(drop=True)], axis=1)
        no_samp_data_scaled.to_csv(f'{featurized_filepath}{nek}_{feat}_none_scaled.csv',index=False)

# MFP feats are inherently scaled
for nek in neks: 
   for feat in ['MFP']: 
      
        df = pd.read_csv(f'{featurized_filepath}{nek}_{feat}_none_unscaled.csv')
        df.to_csv(f'{featurized_filepath}{nek}_{feat}_none_scaled.csv',index=False)


In [ ]:
id_cols = ['NEK', 'compound_id', 'base_rdkit_smiles','subset', 'active'] 
def over_sampling(data_path,filename, sampling):
    """Oversample the datasetes using the SMOTE or ADASYN
    Keeps the feature names and id cols
    file_name (full/absolute path): use the scaled dataframe we just created above 'NEK#_(binding/inhibition)_(MOE/MFP)_none_scaled_df.csv'
    sampling (str): 'SMOTE' or 'ADASYN'
    returns: oversampled dataframe
    """
    
    df = pd.read_csv(data_path+filename) # this is the already scaled ver
    feat_cols = list(set(list(df.columns))-set(id_cols))
    
    # train and test 
    train = df[df['subset']=='train'] 
    test =df[df['subset']=='test'] 
    nek = df['NEK'].iloc[0]
    print(f"nek: {nek}, {type(nek)}")
    # separate just id cols
    just_ids = ['NEK', 'compound_id', 'base_rdkit_smiles','subset']
    train_just_ids = train[just_ids]
    test_just_ids = test[just_ids]

    # just feats and 'active'
    trainX = train[feat_cols]
    testX = test[feat_cols]
    
    trainy = train['active']
    testy = test['active']
    
    if sampling == 'ADASYN':
        oversample = ADASYN(random_state=42)
    else: 
        oversample = SMOTE(random_state=42)

    
    trainX_temp, trainy_temp = oversample.fit_resample(trainX.to_numpy(), trainy.to_numpy().reshape(-1))
    print(f'train after {sampling}: {trainX_temp.shape}')
    
    trainX_resamp = pd.DataFrame(trainX_temp, columns=feat_cols)
    trainy_resamp = pd.DataFrame(trainy_temp, columns=['active'])

    num_real = len(train)
    num_synthetic = len(trainX_resamp)-num_real
    synthetic_ids = pd.DataFrame({'NEK': [nek] * num_synthetic,
        'compound_id': [f'synthetic_{sampling}_{i}' for i in range(num_synthetic)],
        'base_rdkit_smiles': [f'synthetic_{sampling}'] * num_synthetic,
        'subset': ['train']*num_synthetic}) # ,'active':[1]*num_synthetic}

    real_ids = train_just_ids.reset_index(drop=True)
    combined_ids = pd.concat([real_ids,synthetic_ids], ignore_index=True)
    
    train_resamp = pd.concat([combined_ids, trainX_resamp, trainy_resamp[['active']]], axis=1)

    print(train_resamp.columns[train_resamp.columns.duplicated()])
    test_df_final = pd.concat([test_just_ids.reset_index(drop=True),
                               testX.reset_index(drop=True), testy.reset_index(drop=True)],axis=1)

    
    
    # test_df_final['subset'] = 'test'
    # train_resamp = train_resamp.reindex(columns=df.columns)
    # test_df_final = test_df_final.reindex(columns=df.columns)
    
    
    final_df = pd.concat([train_resamp, test_df_final]).reset_index(drop=True)


    return final_df[list(df.columns)]

In [ ]:
#over_sampling(featurized_filepath, 'NEK2_binding_MFP_none_scaled.csv', "SMOTE", "NEK2_binding")
#testing_jaycee = over_sampling('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/', 'NEK2_binding_MOE_none.csv', "SMOTE", "NEK2_binding")
testing_rs = over_sampling('/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/', 'NEK2_binding_MFP_none_scaled.csv', "SMOTE")

print(testing_rs['subset'].value_counts())
print(testing_rs[testing_rs['subset']=='train']['active'].value_counts())
print(testing_rs[testing_rs['subset']=='test']['active'].value_counts())
testing_rs[testing_rs["base_rdkit_smiles"] == "synthetic_SMOTE"]


In [ ]:
id_cols = ['NEK', 'compound_id', 'base_rdkit_smiles','subset', 'active'] 
from imblearn.under_sampling import RandomUnderSampler
def under_sampling(path,filename): 
    df = pd.read_csv(path+filename) # this is the already scaled ver
    feat_cols = list(set(list(df.columns))-set(id_cols))
    
    # train and test 
    train = df[df['subset']=='train'] 
    test =df[df['subset']=='test'] 

    # separate just id cols
    just_ids = ['NEK', 'compound_id', 'base_rdkit_smiles','subset']
    train_just_ids = train[just_ids]
    test_just_ids = test[just_ids]

    # just feats and 'active'
    trainX = train[feat_cols]
    testX = test[feat_cols]
    
    trainy = train['active']
    testy = test['active']
    
    undersample = RandomUnderSampler(random_state=42)
    
    trainX_temp, trainy_temp = undersample.fit_resample(trainX.to_numpy(), trainy.to_numpy().reshape(-1))
    
    trainX_resamp = pd.DataFrame(trainX_temp, columns=feat_cols)
    trainy_resamp = pd.DataFrame(trainy_temp, columns=['active'])
    
    train_ids_resamp = train_just_ids.iloc[trainX_resamp.index].reset_index(drop=True)
    train_resamp= pd.concat([train_ids_resamp, trainX_resamp,trainy_resamp], axis=1)
    # train_resamp['subset'] = 'train'

    test_df_final = pd.concat([test_just_ids.reset_index(drop=True),testX.reset_index(drop=True),testy.reset_index(drop=True)],axis=1)
    # test_df_final['subset'] = 'test'
    final_df = pd.concat([train_resamp,test_df_final]).reset_index(drop=True)
    return final_df[list(df.columns)]
    

In [ ]:
testing_rs_under = under_sampling('/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/', 'NEK2_binding_MFP_none_scaled.csv')

print(testing_rs_under['subset'].value_counts())
print(testing_rs_under[testing_rs_under['subset']=='train']['active'].value_counts())
print(testing_rs_under[testing_rs_under['subset']=='test']['active'].value_counts())

In [ ]:
id_cols = ['NEK', 'compound_id', 'base_rdkit_smiles','subset', 'active'] ### MAKE SURE FOLD IS NOT IN THIS 
filepath = '/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/'

def create_arrays(filepath, root,save_path, printout=True): 
    df = pd.read_csv(filepath+root+".csv")
    train = df[df['subset']=='train']
    test=df[df['subset']=='test']
    feat_cols = list(set(list(df.columns))-set(id_cols))
    
    train_y = train['active'].to_numpy().reshape(-1)
    test_y=test['active'].to_numpy().reshape(-1)
    trainX = train.drop(columns=id_cols).to_numpy()
    testX = test.drop(columns=id_cols).to_numpy()
    if printout: 
        print(f'train X shape: {trainX.shape}, y: {train_y.shape}, test X: {testX.shape}, y:{test_y.shape}')
    if save_path is not None: 
        trainX = pd.DataFrame(trainX)
        trainX.to_csv(filepath+root+'_trainX.csv', index=False)
   
        trainy_df = pd.DataFrame(train_y)
        trainy_df.to_csv(filepath+root+'_train_y.csv', index=False) 

        testX = pd.DataFrame(testX).iloc[:,:-1]
        testX.to_csv(filepath+root+'_testX.csv', index=False)

        testy_df = pd.DataFrame(test_y)
        testy_df.to_csv(filepath+root+'_test_y.csv', index=False) 
        
    return trainX, train_y, testX, test_y

In [ ]:
filepath = '/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/'
feats = ["MOE", "MFP"]
for nek in neks: 
    for feat in feats: 
        for samp in ['UNDER', 'SMOTE', 'ADASYN', "none_scaled"]:
            og_df_name = f'{nek}_{feat}_none_scaled.csv'
            #print(f"feat:{feat}, samp:{samp}, nek:{nek}")
            if samp == 'UNDER': 
                sampled_df = under_sampling(filepath,og_df_name) 
                sampled_df.to_csv(f'{filepath}{nek}_{feat}_{samp}.csv',index=False)
                print(f"feat:{feat}, samp:{samp}, nek:{nek}")
            elif samp == "SMOTE" or samp == "ADASYN": 
                sampled_df=over_sampling(filepath,og_df_name, samp) 
                sampled_df.to_csv(f'{filepath}{nek}_{feat}_{samp}.csv',index=False) 
                #print(f"feat:{feat}, samp:{samp}, nek:{nek}")    
            root_name = f'{nek}_{feat}_{samp}'
            _, _,_,_=create_arrays(filepath, root_name,filepath)

In [ ]:
def create_folds_train(datapath, filename, k_splits): 
    """filename: NEK#_1_uM_min_50_pct_(binding or inhibition).csv"""
    #df = remove_duplicates(datapath, filename)
    df = pd.read_csv(f'{datapath}{filename}')
    df = df[df["subset"]== "train"]


    df['source'] = df['base_rdkit_smiles'].apply(lambda x: 'synthetic' if 'synthetic' in str(x).lower() else 'real')
    df['stratify_key'] = df['active'].astype(str) + "_" + df['source']

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    df["fold"] = -1
    
    for i, (train_index, test_index) in enumerate(skf.split(df, df['stratify_key'])):
        df.loc[df.index[test_index],'fold'] = f"fold{i+1}"
    
    remove_cols = ["source", 'stratify_key']
    df = df.drop(columns=remove_cols)
        
    #display(df)
    return df

In [ ]:
#create_folds_train('/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/', 'NEK2_binding_MFP_ADASYN.csv',5)

featurized_path = '/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/'
feats = ["MOE", "MFP"]
for nek in neks: 
    for feat in feats: 
        for samp in ['UNDER', 'SMOTE', 'ADASYN', "none_scaled"]:
            og_df_name = f'{nek}_{feat}_{samp}.csv'
            folded_df_val = create_folds_train(featurized_path, og_df_name,5)
            type_df = type(folded_df_val)
            folded_df_val.to_csv('/Users/radhi/Desktop/GitHub/atom2024/atom2024/notebooks/paper/datasets/80train_20test/k_fold/'+f'{nek}_{feat}_{samp}_kfold.csv', index=False)